In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from sklearn.model_selection import RandomizedSearchCV

In [0]:
#import dataset
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
y = pd.read_csv('labels.csv')

In [0]:
#label encode all object type features
#bin the age

In [54]:
data_test['age'].unique()

array([ 20,  25,   5,   0,  15,  55,  35,  30,  40,  70,  60,  45,  90,
        10, 100,  65,  50, 995,  80,  75,  95,  85, 110, 200, 105, 150,
       120, 180, 115, 125, 130, 160, 155, 140, 190, 170])

In [0]:
#bining age
bins = [-1,30,60,90,120,150,185,201,995]
labels = [1,2,3,4,5,6,7,8]
data['age_bin'] = pd.cut(data['age'],bins,labels=labels)
data_test['age_bin'] = pd.cut(data_test['age'],bins,labels=labels)

In [0]:
#drop redandunt feature
data = data.drop(columns=['building_id','age'],axis=1)
data_test = data_test.drop(columns=['building_id','age'],axis=1)

In [0]:
#labelencode ordinal
label_encoder_feature = ['land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type','position','plan_configuration','legal_ownership_status','age_bin']
for i in label_encoder_feature :
  labelencoder = LabelEncoder()
  labelencoder.fit(data[i])
  data[i] = labelencoder.transform(data[i])
  data_test[i] = labelencoder.transform(data_test[i])

In [0]:
x_train = data.values
x_test = data_test.values

In [0]:
#scaling features
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [0]:
#xg-boost
model = XGBClassifier()
model.fit(x_train, y)

In [0]:
#svm model
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train,y)

In [71]:
#logistic model
lgmodel = LogisticRegression(C=10, penalty='l2')
lgmodel.fit(x_train,y)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
#used ANN
y = tf.keras.utils.to_categorical(y['damage_grade'])
y = np.delete(y, 0, 1)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128,kernel_initializer='uniform',activation='relu',input_dim=38))
model.add(tf.keras.layers.Dense(128,kernel_initializer='uniform',activation='relu'))
model.add(tf.keras.layers.Dense(3,kernel_initializer='uniform',activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y,epochs=100,batch_size=32)

In [0]:
#for svm and xg boost
y_pred = lgmodel.predict(x_test)
sub = pd.read_csv('sub.csv')
sub['damage_grade'] = y_pred
sub.to_csv('areana.csv',index=False)

In [0]:
#for ann
y_pred = model.predict(x_test)
pred = []
for i in y_pred :
  pred.append(np.argmax(i))
sub = pd.read_csv('sub.csv')
sub['damage_grade'] = pred
sub.to_csv('finalsss.csv',index=False)

In [0]:
#hyper-parameter tuning
hyperparams = {
  'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
  'penalty' : ['l1', 'l2']
}
estimator = LogisticRegression() 

random = RandomizedSearchCV(estimator,hyperparams,scoring='accuracy',cv=10)
random_search=random.fit(x_train,y)
best_result = random_search.best_score_
best_parameters = random_search.best_params_